<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Visión por Computadora</center></span>
## <span style="color:red"><center>Inception y DenseNet: Clasificación Moderna</center></span>

##   <span style="color:blue">Profesores</span>

### Coordinador


- Campo Elías Pardo Turriago, cepardot@unal.edu.co 

### Conferencistas


- Alvaro  Montenegro, PhD, ammontenegrod@unal.edu.co
- Daniel  Montenegro, Msc, dextronomo@gmail.com 
- Oleg Jarma, Estadístico, ojarmam@unal.edu.co 

## <span style="color:blue">Asesora Medios y Marketing digital</span>

- Maria del Pilar Montenegro, pmontenegro88@gmail.com 



## <span style="color:blue">Asistentes</span>



- Nayibe Yesenia Arias, naariasc@unal.edu.co
- Venus Celeste Puertas, vpuertasg@unal.edu.co 

## <span style="color:blue">Introducción</span>

En el año 1989 Se propone la estructura LeNet5, considerada la primera red neuronal convolucional. 

<figure>
<center>
<img src="../Imagenes/LeNet.png" width="700" height="200" align="center"/> 
</center>
</figure>

Este tipo de arquitecturas fueron "muy avanzados" para la época, siendo incapaces de encontrar buenos métodos de aprendizaje con la tecnología del momento.

En 2010 se inicia el "ImageNet Large Scale visual Recognition Challenge"(ILSVRC), un concurso para promover la mejora en las tareas de clasificación

En 2012, en la tercera edición del reto, Apareció AlexNet, que trajo el diseño de LeNet al siglo XXI. Ganando por una diferencia considerable

<figure>
<center>
<img src="../Imagenes/alexnet-vs-lenet.png" width="600" height="450" align="center"/> 
</center>
</figure>

En 2014, la arquitectura VGG construyó sobre alexnet, haciéndola más grande y más poderosa. Aquí nació el termino de "bloques", grupos de capas con una estructura determinada, logrando agregar más capas convolucionales. Aunque se puede decir que este tipo de red fue la que más inspiración ha generado, no fue la verdadera ganadora.

<figure>
<center>
<img src="../Imagenes/vgg-vs-alexnet.png" width="450" height="600" align="center"/> 
</center>
</figure>

En 2015, buscando superar el número de capas de VGG, nacen los bloques residuales y ResNet, la cual [hemos hablado más a fondo](../Cuadernos/redes_residuales.ipynb)

<figure>
<center>
<img src="../Imagenes/vgg-vs-resnet.png" width="1100" height="500" align="center"/> 
</center>
</figure>

Antes de cambiar de tema, es importante revisar ciertos modelos que no siguieron necesariamente la filosofía de AlexNet. Y al mismo tiempo, vamos a ver las estructuras que se construyeron a partir de ResNet

## <span style="color:blue">GoogLeNet y los bloques Inception</span>

Supongamos, por un momento, que estamos en 2014. Aún no tenemos los avances de VGG y ResNet. ¿Cuáles son los obstáculos a superar en las redes convolucionales profundas?

Primero, para hacer un clasificador robusto, es necesario pensar en la variabilidad de tamaños de los objetos a encontrar. 

<figure>
<center>
<img src="../Imagenes/dogs_different_areas.png" width="800" height="300" align="center"/> 
</center>
</figure>

Todos estos perros tienen que ser identificados, a pesar de las diferencias en posiciones y tamaños relativos a la imagen. Ya hemos visto técnicas como [MTCNN](../Cuadernos/rostros_cnn.ipynb) y la "pirámide de imágenes", pero ¿Qué tal si, en lugar de variar el tamaño de la imagen, variaramos los tamaños de los kernel?

Segundo, Según se ha visto en la teoría, agregar más capas a la red puede causar sobreajuste. Así que, qué tal si aplicaramos capas en el mismo nivel, de manera simultanea?

Por último, trabajar con múltiples canales (como son imágenes a color) y grandes tamaños de Kernel es computacionalmente costo. ¿Habrá alguna forma de resumir esta información?




### <span style="color:blue">Inception ingenuo</span>

Enfoquémonos en los dos primeros obstáculos.

La proposición inicial de los bloques Inception era (recordando el nombre) "Redes dentro de Redes".

<figure>
<center>
<img src="../Imagenes/naive_inception.png" width="800" height="400" align="center"/> 
</center>
</figure>


Como dijimos anteriormente, en lugar de obtener diferentes tamaños de la imagen, para que un mismo kernel encuentre diferentes tamaños en esta, vamos a pasar la misma imagen sobre kernel de distintos tamaños, de tal forma que se encuentren los objetivos sin importar el tamaño. Luego se concatenan los resultados, uniendo todo lo aprendido.

Todo esto sucede al mismo tiempo, Necesitando relativamente menos capas de manera secuencial, evitando el sobreajuste

### <span style="color:blue">Inception con reducción de dimensiones</span>

A pesar de tener "menos" capas, aún estamos aplicando varios procesos computacionales. De hecho, podemos pensar que el costo es mayor debido a la cantidad de procesos simultáneos. Así que es necesario reducir esto.

La forma que encontraron de lograr esto es a partir de la reducción de dimensiones con kernel 1x1.

En los caminos de kernel 3x3 y 5x5, se aplica con anterioridad este pequeño Kernel. Se pensaría que esto no reduciría nada, ya que se está haciendo un análisis pixel por pixel, pero lo que hacemos es enfocarnos en una dimensión diferente. Usualmente, la red busca la información general sobre regiones de la imagen sobre cada canal por separado. En este caso, buscamos la información de la imagen sobre todos los canales a la vez. 
 Así los datos a procesar por los kernel es, por lo menos, 3 veces menor. 

<figure>
<center>
<img src="../Imagenes/full_inception.png" width="850" height="400" align="center"/> 
</center>
</figure>

GoogLeNet no es más que varios de estos bloques conectados, con average pooling al final.

<figure>
<center>
<img src="../Imagenes/googlenet.png" width="1200" height="400" align="center"/> 
</center>
</figure>

Vamos a hacer una implementación desde 0. (PELIGRO, CORRER ÚNICAMENTE EN COLAB)

In [1]:
import os
import numpy as np
import random
from types import SimpleNamespace

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim

import torchvision
from torchvision import models
from torchvision.datasets import CIFAR10
from torchvision import transforms

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
import urllib.request
from urllib.error import HTTPError
# URL de github donde están los modelos pre entrenados
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial5/"
# Archivos a descargar
pretrained_files = ["GoogleNet.ckpt", "DenseNet.ckpt"]
# Crear directorio de checkopoints si no existe
os.makedirs('../modelos', exist_ok=True)

# Revisar si cada archivo ya existe, y descargarlo si no
for file_name in pretrained_files:
    file_path = os.path.join('../modelos', file_name)
    if "/" in file_name:
        os.makedirs(file_path.rsplit("/",1)[0], exist_ok=True)
    if not os.path.isfile(file_path):
        file_url = base_url + file_name
        print(f"Descargando {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
        except HTTPError as e:
            print("Errores encontrados durante la descarga:\n", e)

In [3]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = CIFAR10(root='/mnt/storage/Datasets', transform=transform, train=True, download=True)
test_dataset = CIFAR10(root='/mnt/storage/Datasets', transform=transform, train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
train_set, val_set = torch.utils.data.random_split(train_dataset, [45000, 5000])

train_loader = data.DataLoader(train_set, batch_size=128, shuffle=True, pin_memory=True, num_workers=4)
val_loader = data.DataLoader(val_set, batch_size=128, shuffle=False, num_workers=4)
test_loader = data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

In [5]:
import pytorch_lightning as pl

pl.seed_everything(13)

Global seed set to 13


13

In [6]:
class CIFARModule(pl.LightningModule):
    
    def __init__(self, model_name, model_hparams, optimizer_name, optimizer_hparams):
        
        super().__init__()
        self.save_hyperparameters()
        self.model = create_model(model_name, model_hparams)
        self.loss_module = nn.CrossEntropyLoss()
        
    def forward(self, imgs):
        return self.model(imgs)
    
    def configure_optimizers(self):
        if self.hparams.optimizer_name == 'Adam':
            optimizer = optim.AdamW(
                self.parameters(), **self.hparams.optimizer_hparams)
        elif self.hparams.optimizer_name == "SGD":
            optimizer = optim.SGD(self.parameters(), **self.hparams.optimizer_hparams)
        else:
            assert False, f"Optimizador desconocido: \"{self.hparams.optimizer_name}\"" 
            
        scheduler = optim.lr_scheduler.MultiStepLR(
            optimizer, milestones=[100, 150], gamma=0.1)
        return [optimizer], [scheduler]
    
    def training_step(self, batch, batch_idx):
        # "batch" es la salida del dataloader de entrenamiento.
        imgs, labels = batch
        preds = self.model(imgs)
        loss = self.loss_module(preds, labels)
        acc = (preds.argmax(dim=-1) == labels).float().mean()

        self.log('train_acc', acc, on_step=False, on_epoch=True)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        preds = self.model(imgs).argmax(dim=-1)
        acc = (labels == preds).float().mean()
        self.log('val_acc', acc)
        
    def test_step(self, batch, batch_idx):
        imgs, labels = batch
        preds = self.model(imgs).argmax(dim=-1)
        acc = (labels == preds).float().mean()
        self.log('test_acc', acc)

In [7]:
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

model_dict = {}

def create_model(model_name, model_hparams):
    if model_name in model_dict:
        return model_dict[model_name](**model_hparams)
    else:
        assert False, f"Modelo desconocido \"{model_name}\". Los modelos disponibles son: {str(model_dict.keys())}"
        
act_fn_by_name = {
    "tanh": nn.Tanh,
    "relu": nn.ReLU,
    "leakyrelu": nn.LeakyReLU,
    "gelu": nn.GELU
}

In [8]:
def train_model(model_name, save_name=None, **kwargs):

    if save_name is None:
        save_name = model_name

    # Se crea el entrenador de lightning
    trainer = pl.Trainer(default_root_dir=os.path.join('../modelos/', save_name),                            # Donde se guardan los modelos
                         gpus=1 if str(device)=="cuda:0" else 0,                                             # Corremos en una sola GPU(si es posible)
                         max_epochs=180,                                                                     # número de epocas a entrenar si no hay un límite de paciencia
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc"),  # Se guarda el mejor checkpoint basado en la precisión de validación. Se guardan los pesos
                                    LearningRateMonitor("epoch")])                                           # Se crean logs de la rata de aprendizaje

    # Se revisa si existe un modelo pre entrenado existe. En caso de si, se salta el entrenamiento
    pretrained_filename = os.path.join('../modelos/', save_name + ".ckpt")
    if os.path.isfile(pretrained_filename):
        print(f"Modelo encontrado en {pretrained_filename}, cargando...")
        model = CIFARModule.load_from_checkpoint(pretrained_filename) # Se carga el modelo con los hiperparámetros
    else:
        pl.seed_everything(42) # Para mantener reproductibilidad
        model = CIFARModule(model_name=model_name, **kwargs)
        trainer.fit(model, train_loader, val_loader)
        model = CIFARModule.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Se carga el mejor checkpoint después del entrenamiento

    # Se revisa el mejor modelo en los datos de validación y prueba
    val_result = trainer.test(model, test_dataloaders=val_loader, verbose=False)
    test_result = trainer.test(model, test_dataloaders=test_loader, verbose=False)
    result = {"test": test_result[0]["test_acc"], "val": val_result[0]["test_acc"]}

    return model, result

In [9]:
class InceptionBlock(nn.Module):

    def __init__(self, c_in, c_red : dict, c_out : dict, act_fn):
        """
        Inputs:
            c_in - número de mapas de características de las capas anteriores
            c_red - diccionario con llaves "3x3" and "5x5" que especifican la salida de los kernel 1x1
            c_out - diccionario con llaves "1x1", "3x3", "5x5", y "max"
            act_fn - función de activación a usar (e.g. nn.ReLU)
        """
        super().__init__()

        # rama de convolución 1x1 
        self.conv_1x1 = nn.Sequential(
            nn.Conv2d(c_in, c_out["1x1"], kernel_size=1),
            nn.BatchNorm2d(c_out["1x1"]),
            act_fn()
        )

        # rama de convolución 3x3
        self.conv_3x3 = nn.Sequential(
            nn.Conv2d(c_in, c_red["3x3"], kernel_size=1),
            nn.BatchNorm2d(c_red["3x3"]),
            act_fn(),
            nn.Conv2d(c_red["3x3"], c_out["3x3"], kernel_size=3, padding=1),
            nn.BatchNorm2d(c_out["3x3"]),
            act_fn()
        )

        # rama de convolución 5x5
        self.conv_5x5 = nn.Sequential(
            nn.Conv2d(c_in, c_red["5x5"], kernel_size=1),
            nn.BatchNorm2d(c_red["5x5"]),
            act_fn(),
            nn.Conv2d(c_red["5x5"], c_out["5x5"], kernel_size=5, padding=2),
            nn.BatchNorm2d(c_out["5x5"]),
            act_fn()
        )

        # rama Max-pool
        self.max_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, padding=1, stride=1),
            nn.Conv2d(c_in, c_out["max"], kernel_size=1),
            nn.BatchNorm2d(c_out["max"]),
            act_fn()
        )

    def forward(self, x):
        x_1x1 = self.conv_1x1(x)
        x_3x3 = self.conv_3x3(x)
        x_5x5 = self.conv_5x5(x)
        x_max = self.max_pool(x)
        x_out = torch.cat([x_1x1, x_3x3, x_5x5, x_max], dim=1)
        return x_out

In [10]:
class GoogleNet(nn.Module):

    def __init__(self, num_classes=10, act_fn_name="relu", **kwargs):
        super(GoogleNet, self).__init__()
        self.hparams = SimpleNamespace(num_classes=num_classes,
                                       act_fn_name=act_fn_name,
                                       act_fn=act_fn_by_name[act_fn_name])
        self._create_network()
        self._init_params()

    def _create_network(self):
        #Primera convolución de la imagen original para escalar al tamaño del canal
        self.input_net = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            self.hparams.act_fn()
        )
        # Se apilan los bloques inception
        self.inception_blocks = nn.Sequential(
            InceptionBlock(64, c_red={"3x3": 32, "5x5": 16}, c_out={"1x1": 16, "3x3": 32, "5x5": 8, "max": 8}, act_fn=self.hparams.act_fn),
            InceptionBlock(64, c_red={"3x3": 32, "5x5": 16}, c_out={"1x1": 24, "3x3": 48, "5x5": 12, "max": 12}, act_fn=self.hparams.act_fn),
            nn.MaxPool2d(3, stride=2, padding=1),  # 32x32 => 16x16
            InceptionBlock(96, c_red={"3x3": 32, "5x5": 16}, c_out={"1x1": 24, "3x3": 48, "5x5": 12, "max": 12}, act_fn=self.hparams.act_fn),
            InceptionBlock(96, c_red={"3x3": 32, "5x5": 16}, c_out={"1x1": 16, "3x3": 48, "5x5": 16, "max": 16}, act_fn=self.hparams.act_fn),
            InceptionBlock(96, c_red={"3x3": 32, "5x5": 16}, c_out={"1x1": 16, "3x3": 48, "5x5": 16, "max": 16}, act_fn=self.hparams.act_fn),
            InceptionBlock(96, c_red={"3x3": 32, "5x5": 16}, c_out={"1x1": 32, "3x3": 48, "5x5": 24, "max": 24}, act_fn=self.hparams.act_fn),
            nn.MaxPool2d(3, stride=2, padding=1),  # 16x16 => 8x8
            InceptionBlock(128, c_red={"3x3": 48, "5x5": 16}, c_out={"1x1": 32, "3x3": 64, "5x5": 16, "max": 16}, act_fn=self.hparams.act_fn),
            InceptionBlock(128, c_red={"3x3": 48, "5x5": 16}, c_out={"1x1": 32, "3x3": 64, "5x5": 16, "max": 16}, act_fn=self.hparams.act_fn)
        )
        # sección completamente conectada
        self.output_net = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(128, self.hparams.num_classes)
        )

    def _init_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(
                    m.weight, nonlinearity=self.hparams.act_fn_name)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.input_net(x)
        x = self.inception_blocks(x)
        x = self.output_net(x)
        return x

In [11]:
model_dict["GoogleNet"] = GoogleNet

In [ ]:
googlenet_model, googlenet_results = train_model(model_name="GoogleNet",
                                                 model_hparams={"num_classes": 10,
                                                                "act_fn_name": "relu"},
                                                 optimizer_name="Adam",
                                                 optimizer_hparams={"lr": 1e-3,
                                                                    "weight_decay": 1e-4})

In [ ]:
print("Resultados GoogleNet", googlenet_results)

## <span style="color:blue">Densenet</span>

Recordando las redes residuales, un bloque de esta red da una especie de pista para no tener que encontrar la mejor función y parámetros de la nada. Dando en cierta forma la capacidad de "comparar" con los resultados inmediatamente anteriores.

<figure>
<center>
<img src="../Imagenes/residual-block.svg" width="600" height="400" align="center"/> 
</center>
</figure>

Estas redes dieron una enseñanza importante: Podemos obtener redes más profundas, con mayor precisión y menor tiempo de entrenamiento si logramos hacer pequeñas conexiones entre las capas cercanas a la entrada y la salida. La evolución lógica de esta enseñanza es: Hagan más conexiones, entre más mejor, CONECTENLO TODO

### <span style="color:blue">Bloque Denso</span>

En el bloque residual, se utiliza una suma entre los valores originales de entrada y los datos procesados por las capas convolucionales. Esto se hace para mejorar la convergencia de un bloque al otro.

En este nuevo caso, no queremos "sumar" la información, simplemente capturarlos. Aquí, todos los valores de entrada del bloque serán "concatenados", aumentando el número de características. Todo este proceso es "recursivo", así que las concatenaciones de las capas anteriores serán agregadas a la siguiente. Tendremos cada vez más y más mapas de características a medida que se procesan los bloques.

<figure>
<center>
<img src="../Imagenes/densenet.gif" width="600" height="200" align="center"/> 
</center>
</figure>

Gracias a esto necesitamos menos canales en cada capa. Definimos un parametro "k", que será el número de canales por capa. Vamos a llamarlo la "Rata de crecimiento".

<figure>
<center>
<img src="../Imagenes/growth_rate.png" width="600" height="300" align="center"/> 
</center>
</figure>


### <span style="color:blue">La arquitectura DenseNet</span>

- Capa de transición: 
    - Normalización por lotes
    - Activación ReLU
    - convolución con kernel 1x1
- Capa de composición:
    - Normalización por lotes
    - Activación ReLU
    - convolución con kernel 3x3


<figure>
<center>
<img src="../Imagenes/full_densenet.png" width="800" height="300" align="center"/> 
</center>
</figure>

In [13]:
class DenseLayer(nn.Module):

    def __init__(self, c_in, bn_size, growth_rate, act_fn):
        """
        Inputs:
            c_in - Número de canales de entrada
            bn_size - Tamaño de la boca de botella (factor de Rata de crecimiento) para el kernel 1x1. Usualmente entre 2 y 4
            growth_rate - número de canales para los kernel 3x3
            act_fn - función de activación (e.g. nn.ReLU)
        """
        super().__init__()
        self.net = nn.Sequential(
            nn.BatchNorm2d(c_in),
            act_fn(),
            nn.Conv2d(c_in, bn_size * growth_rate, kernel_size=1, bias=False),
            nn.BatchNorm2d(bn_size * growth_rate),
            act_fn(),
            nn.Conv2d(bn_size * growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)
        )

    def forward(self, x):
        out = self.net(x)
        out = torch.cat([out, x], dim=1)
        return out

In [14]:
class DenseBlock(nn.Module):

    def __init__(self, c_in, num_layers, bn_size, growth_rate, act_fn):
        """
        Inputs:
            c_in - Número de canales de entrada
            num_layers - Número de capas Densas dentro del bloque
            bn_size - Tamaño de la boca de botella para usar en las capas densas
            growth_rate - Rata de crecimiento dentro de las capas densas
            act_fn - Función de activación
        """
        super().__init__()
        layers = []
        for layer_idx in range(num_layers):
            layers.append(
                DenseLayer(c_in=c_in + layer_idx * growth_rate, # Los canales de entrada son los originales y los mapas de características anteriores 
                           bn_size=bn_size,
                           growth_rate=growth_rate,
                           act_fn=act_fn)
            )
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        out = self.block(x)
        return out

In [15]:
class TransitionLayer(nn.Module):

    def __init__(self, c_in, c_out, act_fn):
        super().__init__()
        self.transition = nn.Sequential(
            nn.BatchNorm2d(c_in),
            act_fn(),
            nn.Conv2d(c_in, c_out, kernel_size=1, bias=False),
            nn.AvgPool2d(kernel_size=2, stride=2) 
        )

    def forward(self, x):
        return self.transition(x)

In [16]:
class DenseNet(nn.Module):

    def __init__(self, num_classes=10, num_layers=[6,6,6,6], bn_size=2, growth_rate=16, act_fn_name="relu", **kwargs):
        super().__init__()
        self.hparams = SimpleNamespace(num_classes=num_classes,
                                       num_layers=num_layers,
                                       bn_size=bn_size,
                                       growth_rate=growth_rate,
                                       act_fn_name=act_fn_name,
                                       act_fn=act_fn_by_name[act_fn_name])
        self._create_network()
        self._init_params()

    def _create_network(self):
        c_hidden = self.hparams.growth_rate * self.hparams.bn_size # The start number of hidden channels

        # primera convolución para cambiar el tamaño
        self.input_net = nn.Sequential(
            nn.Conv2d(3, c_hidden, kernel_size=3, padding=1)  # no se hace normalización de lotes ni función de activación
           
        )

        # Se crean los bloques densos, eventualmente agregando las capas de transición
        blocks = []
        for block_idx, num_layers in enumerate(self.hparams.num_layers):
            blocks.append(
                DenseBlock(c_in=c_hidden,
                           num_layers=num_layers,
                           bn_size=self.hparams.bn_size,
                           growth_rate=self.hparams.growth_rate,
                           act_fn=self.hparams.act_fn)
            )
            c_hidden = c_hidden + num_layers * self.hparams.growth_rate #salida general de los bloques densos
            if block_idx < len(self.hparams.num_layers)-1: #  No se aplica transición en el último bloque
                blocks.append(
                    TransitionLayer(c_in=c_hidden,
                                    c_out=c_hidden // 2,
                                    act_fn=self.hparams.act_fn))
                c_hidden = c_hidden // 2

        self.blocks = nn.Sequential(*blocks)

        # Sección completamente conectada
        self.output_net = nn.Sequential(
            nn.BatchNorm2d(c_hidden), 
            self.hparams.act_fn(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(c_hidden, self.hparams.num_classes)
        )

    def _init_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity=self.hparams.act_fn_name)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.input_net(x)
        x = self.blocks(x)
        x = self.output_net(x)
        return x

In [17]:
model_dict["DenseNet"] = DenseNet

In [ ]:
densenet_model, densenet_results = train_model(model_name="DenseNet",
                                               model_hparams={"num_classes": 10,
                                                              "num_layers": [6,6,6,6],
                                                              "bn_size": 2,
                                                              "growth_rate": 16,
                                                              "act_fn_name": "relu"},
                                               optimizer_name="Adam",
                                               optimizer_hparams={"lr": 1e-3,
                                                                  "weight_decay": 1e-4})